# Import Statements

In [ ]:
import serial
import time
import pycont.controller as cont
import logging
from new_era.peristaltic_pump import PeristalticPump
import io
from new_era.peristaltic_pump_network import PeristalticPumpNetwork
from math import pi
import sys
import urx
import logging
import keyboard
import asyncio
from urx.robotiq_two_finger_gripper import Gripper
from UR3e_MotionControl import UR3e_MotionControl
from array import *
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt
from north_c9 import NorthC9
from Locator import *
from ftdi_serial import Serial
from nrc_custom.Ecell_package import ECell
from nrc_custom import Videocapture
from nrc_custom import EMAP_movements
from nrc_custom import Characterization_cell as char_cell
from nrc_custom import Slidehotel as sh
from nrc_custom import CellWashing
from nrc_custom.CCUS_PostProcess import PostProcessing
from nrc_custom.CCUS_Active_Learning_Noise import ActiveLearning
import time
import threading
from alicat import FlowController
import nest_asyncio
#need to add this, prevents asyncio from running in the background - otherwise you will have an error on calling this asyncio func.
nest_asyncio.apply()

In [ ]:
# All user defined information:
pump_port = 'COM6'  # Peristaltic pump/syringe pump port check on your own system
valco_valve_port = 'COM13' # Valco valve com port
n9_port = 'COM10' # N9 com port
stepper_slider_port = 'COM4' # Performance cell com port
flow_controller_port = 'COM12' # Flow controller port
# ECell com_port

In [ ]:
# Seem to have a big problem when putting the dispense class as an external library and 'c9' not being defined. 
# Calling the class here seems fine, but as an external library, run into the problem????
# **********************************************************************************************
# Define statements

CAROUSEL_ROT = 4 # rotary
CAROUSEL_Z = 6 # elevation
# **********************************  FLUID DISPENSATION FUNCTIONS END ****************************************

# **********************************************************************************************
# Class 
class DispenseProcedure:
    # **********************************************************************************************
    # Constructor definition
    def __init__(self,pumpnum,target_wgt,carouselsink,carouseldump):
        
        self.pumpnum = pumpnum # 9
        self.target_wgt = target_wgt
        
        self.carouselsink = carouselsink #3 - Count from 1.  Sink position changes depending on which sink port we want to dispense.
        self.carouseldump = carouseldump #2 - Count from 1.  Dump position does not change 

    # **********************************************************************************************
    # Function definition
    
    # Desc : Reports the settings going into the object made by the Constructor
    def ReportSettings(self):
        print("Object Created : Pump Number {0} is active and Target Weight {1} is requested.".format(self.p1.pumpnum,self.p1.target_wgt))

    # Desc : Moves carousel's axes (rotation and elevation) as defined by incoming variables
    def move_carousel(self, rot_deg, z_mm, vel=None, accel=None):
        #self.rot_deg = rot_deg
        #self.z_mm = z_mm
        #self.vel = vel
        #self.accel = accel
        
        if ((rot_deg > 330) or (z_mm > 160)):
            return
        c9.move_axis(CAROUSEL_Z, 0, vel, accel)
        c9.move_axis(CAROUSEL_ROT, int(rot_deg*(51000/360)), vel, accel)
        c9.move_axis(CAROUSEL_Z, int(z_mm*(40000/160)), vel, accel) # vel = counts/sec, accel = counts/sec2

    # Desc : Obtain stable weight of liquid in vial.  Note - mass balance has to be zeroed first.
    def measure_weight(self):
        c9.clear_scale()
        c9.delay(2) # delay enables any drops travelling down the tube fall into the vial
        st = c9.read_steady_scale()
        print(st)
        index = 0
        weight = 0
        
        c9.delay(2)
        weight = st
            
        print("\nFinal stable weight : ", weight)
        return weight

    # Desc : Rotates carousel port to positioned defined by incoming variable.
    #        Checks should be added to function below and tested to ensure port numbers range between 1 and 7 (i.e. not out of range)
    def set_carousel_port(self, pos):
        # pos represents the position of the carousel dispenser from 1 to 7
        self.move_carousel((pos * 45) + 3, 127) # note : the +3 is for the azimuth offset error.  (max vals are 330.0 and 155)
        
    # Desc : Returns carousel axes positions to its home.
    def home_carousel_axis(self):
        # base - rotary (4)
        # top - up/down aka elevation (6)
        c9.home_axis(4)
        c9.home_axis(6)

    # Desc : Clears and zeroes the mass balance.    
    def zero_weigh_scale(self):
        c9.delay(1)
        c9.clear_scale()
        c9.zero_scale()
    
#     def catalyst_procedure(self): # blank holder for catalyst procedure
#         pass

# **********************************************************************************************
    def prime_pumps(self):
        # pos represents the position of the carousel dispenser from 1 to 7
        p1.set_carousel_port(p1.carouseldump) 

        # --------------------------------------------------------------------
        # Prime the pump - this is on the Source side

        # First, set the pump and valve to the default valve position
        # NOTE : Default valve position has the valve to the source tank open.
        c9.set_pump_valve(p1.pumpnum,0)

        c9.delay(1)
        c9.home_pump(p1.pumpnum)

        # home the pump (again?!)
        c9.delay(1)
        c9.set_pump_valve(p1.pumpnum,0)

        # suck up X ml from vial
        c9.delay(1)
        c9.aspirate_ml(p1.pumpnum,1) # 1 was 0.5
        c9.delay(2) # almost certainly need this delay for the fluid to be sucked up fully with the negative pump pressure

        # set the pump and switch the valve to the dispense position
        c9.set_pump_valve(p1.pumpnum,1)

        # dispense X ml from vial
        c9.delay(1)
        c9.dispense_ml(p1.pumpnum,1)
        c9.delay(2) # need this delay as there are still some drops falling as the tube dispenses the fluid with positive pump pressure

        # set the pump and switch the valve back to default valve position
        c9.set_pump_valve(p1.pumpnum,0)
        c9.delay(1)
        
    def catalyst_procedure(self, dispense_num):
        # Move Carousel to position where it will Dispense into Vial
        # dispense_num is for tracking and recording the weights
        p1.set_carousel_port(p1.carouselsink)

        # --------------------------------------------------------------------
        # Have the pump suck up a full cylinder of fluid from Source

        # First, set the pump and valve to the default valve position (just in case)
        # NOTE : Default valve position has the valve to the source tank open.
        c9.set_pump_valve(p1.pumpnum,0)
        c9.delay(1)

        # suck up X ml from source
        c9.aspirate_ml(p1.pumpnum,1) # 1 was 0.5
        c9.delay(2) # almost certainly need this delay for the fluid to be sucked up fully with the negative pump pressure

        # set the pump and switch the valve to the dispense position
        c9.set_pump_valve(p1.pumpnum,1)
        c9.delay(1)

        # -------------------------------------------------------------

        # Zero the weight scale with the empty vial
        # We are about to measure (by weight) how much liquid we have dispensed  
        p1.zero_weigh_scale()

        # -------------------------------------------------------------
        # Measuring weight of (incrementally) dispensed fluid on mass balance in a closed feedback loop manner till it hits the weight target.
        # Dispensation quantity of fluid from the pump is stepped down as mass balance approaches its target of 0.050 mL.
        # For distilled water, there is a 1:1 relationship between fluid weight and fluid volume (i.e. 1.000 g = 1.000 mL).
        # Would need to calculate the ratio for fluids of different chemicals accordingly using their concentration (i.e. molar mass).
        # If the fluid pump has already dispensed more than 3/4 of the quantity of fluid in the cylinder, the pump is refilled (re-primed).

        wgt = p1.measure_weight()
        addon_disp = 0
        # p1.target_wgt = 1.000  # use this if you want to over-ride the setting made in the constructor (way up above)
        dispvar = 0.025  # by default, dispense this much mL (the smallest displacement as seen below in the while)

        # we are attempting to hit the targeted weight within 0.01 of the vial's weight reading
        # whereupon we stop dispensing fluid.
        while ((p1.target_wgt - wgt) > 0.005):
            # dispense X ml from vial    
            if (p1.target_wgt - wgt > 0.50):
                dispvar = 0.45     
            elif (p1.target_wgt - wgt > 0.35):
                dispvar = 0.32   
            elif (p1.target_wgt - wgt > 0.15):
                dispvar = 0.12
            elif (p1.target_wgt - wgt > 0.05):
                dispvar = 0.04
            elif (p1.target_wgt - wgt > 0.02):
                dispvar = 0.025
            else:
                dispvar = 0.025

            addon_disp += dispvar

            c9.dispense_ml(p1.pumpnum,dispvar)

            wgt = p1.measure_weight()
            #print(wgt)

            # if pump has dispensed more than 3/4 of its volume, dump the remainder and refill (re-prime) the pump
            if(addon_disp > 0.9):     
                p1.set_carousel_port(p1.carouseldump) # move to the position of the fluid dump receptical

                c9.delay(1)
                c9.home_pump(p1.pumpnum)

                c9.delay(1)
                c9.set_pump_valve(p1.pumpnum,0)

                # suck up X ml from source
                c9.aspirate_ml(p1.pumpnum,1) # fill full cylinder
                c9.delay(2) # almost certainly need this delay for the fluid to be sucked up fully with the negative pump pressure

                # move carousel back over vial
                p1.set_carousel_port(p1.carouselsink)

                # return back to dispensing valve position
                c9.delay(2)
                c9.set_pump_valve(p1.pumpnum,1)
                c9.delay(2)            

                addon_disp = 0 # reset the adddon_disp variable since we have filled up the cylinder to 100%
                # we are ready to continue

        print(wgt)
        dict[f'{experiment_name}'][f'Test_{exp_count}']['Metric']['X_mass'][dispense_num] = wgt # Add final weight to dictionary for tracking

        # --------------------------------------------------------------------

        # Set the pump and switch the valve back to default valve position.
        c9.delay(0.5)
        c9.set_pump_valve(p1.pumpnum,0)
        c9.delay(2)

        # --------------------------------------------------------------------

        # Move to the fluid dump receptical to dump remainder fluid in the pump's cylinder.
        p1.set_carousel_port(p1.carouseldump)

        # --------------------------------------------------------------------

        # Perform the dump of fluid into the fluid dump receiptical.
        c9.home_pump(p1.pumpnum)
        c9.delay(3)

        # --------------------------------------------------------------------

        # Return the carousel rotation and elevation axis to its home position.
        # p1.home_carousel_axis()

    # --------------------------------------------------------------------
    # END

# Initializing all Connections and Defining Classes

In [ ]:
#Peristaltic pump does not like connecting 2nd, 3rd so connect to it first!

#Connect peristaltic pump and syringe pump for washing
pump_network = PeristalticPumpNetwork(port=pump_port, baudrate=9600)
pump = pump_network.add_pump(address=0, baudrate=9600)  # first pump directly connected to the computer
print("Connected to peristaltic pump")
cell_wash = CellWashing(valve_com_port = valco_valve_port, syringe_com_port = pump_port)

In [ ]:
#Check com everytime! 
#This initializes the emap_movements class (initializes n9 connection when created)
emap = emap_movements(com_port = n9_port)

In [ ]:
 #Initializing the Tri-pump
logging.basicConfig(level=logging.INFO)
setup_config_file = r"C:\Users\nrc-eme-lab\Desktop\sampleconfiguration.json" # check your system
controller = cont.MultiPumpController.from_configfile(setup_config_file)
#initialize the pump - smart initialize is to avoid reinitializing, and resetting the pump back to zero position
controller.smart_initialize()
print("Tri-pump has been connected...")

In [ ]:
# Start a separate thread for each camera
#cameras = [(1, 'camera1'), (2, 'camera2')]
#threads = [CameraCaptureThread(index, filename) for index, filename in cameras]
#for thread in threads:
    #thread.start()

In [ ]:
#Connecting to cell stepper slider

ser = serial.Serial(stepper_slider_port,115200)  # check your own port
print("Stepper slider is connected...")

In [ ]:
#initializing dino-light connection
cap = cv2.VideoCapture(1) # 1 for DinoLite camera (might need to check different values to find the right one
#note: camera indexes will likely change when the other cameras are plugged in as well. Will need to verify the camera index for each 
    
if not cap.isOpened():
    raise IOError("Cannot access the camera - make sure that the port is open/it is plugged in")

(ret, frame) = cap.read() # return a boolean (success flag) and an array corresponding to the color values of the frame you captured
print (ret) # checking the success flag "True"
    #print(frame) #to check the array of the color values for each pixel in the frame

In [ ]:
if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO)
    
    do_wait = True
    if len(sys.argv) > 1:
        do_wait = False

    # establish ethernet connection with robot arm
    rob = urx.Robot("192.168.1.10")
    # create and initialize motion control object
    motion = UR3e_MotionControl(rob, (0.0, 0, 0, 0, 0, 0))

    # TCP in this case is the tool control path.
    print("Setting TCP...")
    rob.set_tcp((0, 0, 0, 0, 0, 0))
    print("Setting Default Maximum Anticipated Payload...")
    # this relates to the maximum anticipated payload expected - weight in kg.
    rob.set_payload(0.5, (0, 0, 0))
    print("Preliminary Settings Complete...")
    
    
    #initializng flow controller:
    flow_controller = FlowController(port= flow_controller_port)
    print("Flow controller is connected...")

In [ ]:
#create an object for the characterization workflow
chara_cell = char_cell.characterization_cell(pump=pump, ser=ser, controller=controller)

In [ ]:
#We only do this once at the beginning
emap.home_pumps()

time.sleep(4)

emap.carousal_startup()

time.sleep(4)

emap.cells_startup()

***Generate the initial data file***

In [ ]:
# This block sets up the initial dictionary as part of the campaign. This is the object that will be updated throughout each run
experiment_name = 'rbnb3pxx'
number_runs = 50
exp_start = 0 # 0 is new campaign, other if continuing an existing campaign
dict = {}
dict[f'{experiment_name}'] = {}

#Pre populate the dictionary with the general data structure
for exp_count in range(number_runs):
    dict[f'{experiment_name}'][f'Test_{exp_count}'] = {'Depo':{},'Char':{},'Metric':{},'AL':{}}

# Create the folder if it doesn't exist based on the date of creation
date_format = date.today().strftime("%Y_%m_%d") # https://www.programiz.com/python-programming/datetime/current-datetime
root_path = f'C:/Users/Blackr/Documents/CCUS/MAPs/Initiate/{date_format}/'

if not os.path.exists(root_path): # Check if folder exists - if not, make it, else use existing folder
    os.makedirs(root_path)

# # Save the initial pickle file
with open(f'{root_path}{experiment_name}_saved_data.pkl', 'wb') as f:
            pickle.dump(dict, f)

***Start the loop here?***

In [ ]:
# The below starts the campaign, pulling in the correct information from the active learning protocol

for exp_count in range(exp_start,number_runs): # Define how many experiments the campaign will be
    # Update test name and dictionary
    test = f'Test_{exp_count}' # current experiment name - note in the potentiostat.py there is built in protection incase forget to change
    filename = f'{experiment_name}_{test}'
    print(f'For checking purposes the file name is: {filename}')

    # Initialize the active learning
    next_experiment = ActiveLearning(root_path, experiment_name, test, exp_count)
    
    # Initialize the experiment_update module
    experiment_update = PostProcessing(root_path, experiment_name, test)
    
    # Pick the first experiment randomly OR follow what was provided:
    if exp_count == 0:
        
        #Call in data from previous method to be able to access here:
        with open(f'{root_path}{experiment_name}_saved_data.pkl', 'rb') as f:
            dict = pickle.load(f)        
            
        next_experiment.determine_first_experiment(dict)
        #print(dict)
        
        X_choice = list(np.array(dict[f'{experiment_name}'][f'Test_{exp_count}']['AL']['X_sample'][0]))
        print(f'Initial experiment to run: {X_choice}')
        
        experiment_update.experiment_update(dict, exp_count, X_choice)
       
        # Set the pump dispension (ie. concentration) amount
        px = []
        px.append(DispenseProcedure(1,X_choice[0],1,0)) # pumpnum,target_wgt,carouselsink,carouseldump
        
    else: # If this is not the first experiment (ie. you need to continue the campaign after an error)
        
        #Call in data from previous method to be able to access here:
        with open(f'{root_path}{experiment_name}_saved_data.pkl', 'rb') as f:
            dict = pickle.load(f)

        #TODO: this will need to pull the last value from the previous test_name based on how it is stored, see Test_{i-1}
        X_choice = list(np.array(dict[f'{experiment_name}'][f'Test_{exp_count-1}']['AL']['X_sample'][-1]))
        
        print(f'Next experiment to run: {X_choice}')
        
        experiment_update.experiment_update(dict, exp_count, X_choice)
        
        # Set the pump dispension (ie. concentration) amount
        px = []
        px.append(DispenseProcedure(1,X_choice[0],1,0)) # pumpnum,target_wgt,carouselsink,carouseldump] # This defines the solution dispense amount

    while True:
        next_exp = input('Understood what experiment to run next? Type -ok- to continue:')
        if next_exp == 'ok':
            break
    
    print("RUN EXPERIMENT")
    print("PROCESS DATA AND UPDATE ASSOCIATED OUTPUT MATRICES WITH NEW DATA")
        
    next_experiment.determine_next_experiment_random(dict)
    
#*************************************************START EXP HERE************************************************
    #initializing the object for the robot movement class
    ur = sh.slidehotel(slide_index=exp_count, motion=motion, rob=rob)
    #UR will pick up a slide from slot_to_use
    ur.pick_slide()
    
    time.sleep(2)

    #we will now close Edep cell
    emap.depo_close()

    #UR will get out of the way
    ur.leave_depo()
    
    #n9 will grab a vial and prepare a solution
    emap.grab_vial() #NEEDS TO INDEX TO EXP_COUNT
    
    emap.fill_vial(px = px)

    time.sleep(2)
    
    #once solution is made, n9 will pipette solution into EDep cell
    emap.pipette_procedure()

    time.sleep(2)
    
    #EDep cell will need to be diluted now
    emap.dilute_depo()
    
    #Now that everything is ready, we are going to start the EDep
    %run pstat_depo.py

    time.sleep(2)
    
    #Now that we ran the Edep we need to wash the cell
    #emap.depo_wash()
    cell_wash.wash_deposition_cell(volume  = 30, num_wash_steps = 3)
    
    #UR will come back now to grip the slide
    ur.return_depo()
    
    time.sleep(2)

    #emap will now open the deposition cell
    emap.open_depo()
    
    time.sleep(2)

    #UR will now carry it to the first characterization station
    ur.depo_to_char1()
    
    time.sleep(2)

    #emap will commence the characterization procedure
    emap.char_procedure()
    
    #Once everything is ready we will run Echem
    %run pstat_char_1.py
    time.sleep(2)
    
    #now we will wash characterization station
    #emap.wash_char()
    cell_wash.wash_characterization_cell(volume  = 30, num_wash_steps = 3)
    time.sleep(2)
    
    #then we will open up the characterization cell
    emap.open_char()
    
    time.sleep(2)

    #UR will now carry it over to the performance characterization cell
    ur.char1_to_char2()
    
    time.sleep(2)

    #closing cell with stepper slider
    chara_cell.startup()
    chara_cell.close_cell()
    
    time.sleep(2)

    #start flowing co2:
    flow_controller.set_flow_rate(20)
    
    #need to prime the system
    chara_cell.tri_priming()
    
    #need to fill cell with electrolyte
    chara_cell.fill_cell()
    
    #let system saturate for 5 minutes
    time.sleep(300)
    
    #now we start Echem
    print("Beginning characterization!")
    %run pstat_char_2.py
    print("Wow I feel extremely characterized")
    
    time.sleep(2)

    #now we remove all electrolyte
    chara_cell.empty_cell()
    
    time.sleep(2)

    #then we will wash the cell
    chara_cell.cell_wash()
    
    time.sleep(2)

    #open cell
    ser = serial.Serial(stepper_slider_port, 115200)  # open serial port
    chara_cell = char_cell.characterization_cell(pump=pump, ser=ser, controller=controller)
    chara_cell.startup()
    chara_cell.open_cell()
    
    time.sleep(2)

    #UR will carry sample to dino-light
    ur.cell_to_dino()
    
    time.sleep(2)

    #we will then take a picture

    chara_cell.takephoto()
    
    time.sleep(2)

    #UR returns the sample
    ur.return_sample()